# Тестирование работы API для тегирования видео

In [ ]:
%pip install pandas httpx --upgrade --quiet

In [1]:
import pandas as pd

## Загружаем тестовую выборку

In [2]:
x_test = pd.read_csv("x_test.csv", index_col=0).reset_index(drop=True)
x_test

,video_id,title,description
0,85c6ac4a4cfc3778cd2e65139d970dba,"Салют, Начальник | 4 серия",На очередной сельской планёрке обсуждают насущ...
1,7eecbd10bdaa596c2921ecb9bf7dbf69,Сколько Стоит Тачка: Галина Ржаксенская - от п...,"В этом выпуске вы узнаете, на чем ездит блогер..."
2,0d7b83b4d141158214acf42ab2ca0519,Какова Меркурия? | Выпуск 12,Женя вместе с астрологом попробуют разобраться...
3,cf3ef0b2d6227ad372a9b7dcb6cb2df3,Смехмашина | Выпуск 13,Подвели итоги розыгрыша. Поздравляем счастливо...
4,c093daa4f0e96d8f2e4736c9240898f3,Абьюз-шоу | Выпуск № 9 часть 1 | Ирина Барышева,Героиней сегодняшнего выпуска Абьюз-шоу стала ...
...,...,...,...
205,3a84cf5e54a272f77470dd066f554edb,Yunan MotoTour в ГрандТуре «Байкальская миля 2...,Бурятский драматический театр в г. Улан-Удэ.
206,484b3600ebf13d5f11b1123b2fe4b142,Роман Юнусов и популярный озвучер Карен Арутюн...,В новом выпуске шоу «Спортивный Интерес» Рома ...
207,d70c45bc09cbbe583c9cede0477040a7,PykoJob | Выпуск 4 | Уютная гостиная с небольш...,Ведущая Алиса и дизайнер Наталья показывают ка...
208,2952b0a7148f5bb09247b22048cd11b9,"Команда ""3/21"" в ГрандТуре-2022: музей В.М. Шу...","""Счастье - оно обыкновенное. Каждый человек ка..."


## Обращаемся к API

In [38]:
import httpx

sample_submission = []
for i, data in x_test.head(2).iterrows():

    with open(f"videos/{data['video_id']}.mp4", "rb") as video:
        files = {'video': video}
        tags = httpx.post(f"https://hack.vopro.site/predict_tokens?title={data['title']}&description={data['description']}", files=files)
    sample_submission.append({
        "video_id": data["video_id"],
        "predicted_tags": tags.text,
    })

sample_submission = pd.DataFrame(sample_submission)
sample_submission.predicted_tags = sample_submission.predicted_tags.apply(lambda x: "[]" if "detail" in x else x)
sample_submission

,video_id,predicted_tags
0,85c6ac4a4cfc3778cd2e65139d970dba,"[""Фильмы и анимация: Комедии"",""Телевидение: Му..."
1,7eecbd10bdaa596c2921ecb9bf7dbf69,"[""Личные финансы"",""Массовая культура: Семьи зн..."


## Загружаем целевые метки

In [29]:
ground_truth = pd.read_csv('y_test.csv', index_col=0)
ground_truth = ground_truth.join(x_test, how='inner')

## Запускаем метрику

In [28]:
import pandas as pd
import argparse
import ast
import numpy as np

# #sample_submission = pd.read_csv('/content/sample_submission.csv')
# sample_submission = pd.DataFrame([['asd', 'name', 'disc', 'tag1']], columns=['video_id', 'title', 'discription', 'tags'])
# sample_submission['predicted_tags'] = np.repeat(["['Карьера: Cтажировки', 'Карьера: Составление резюме']"], len(sample_submission))

def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        print(tag)
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    print()
    return final_tag_list


def find_iou_for_sample_submission(pred_submission, true_submission):
    ground_truth_df = true_submission
    ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
    ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))

    predictions_df = pred_submission
    predictions_df["predicted_tags"] = predictions_df["predicted_tags"].apply(ast.literal_eval)
    predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in ground_truth_df.iterrows():
        predicted_tags = predictions_df[predictions_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter

# ground_truth = pd.DataFrame([['asd']], columns=['video_id'])
# ground_truth['tags'] = np.repeat([["Карьера: Cтажировки, Карьера: Составление резюме"]], len(ground_truth))

if __name__ == '__main__':

    try:
        pred_submission = sample_submission #pd.read_csv(pred_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке решения участника'
    try:
        true_submission = ground_truth #pd.read_csv(true_path, sep = ',')
    except Exception:
        assert False, 'Ошибка при загрузке эталонного решения'


    final_score = find_iou_for_sample_submission(pred_submission, true_submission)
    print("FINAL_SCORE:", final_score) #final_score)

Семья и отношения: Брак и гражданские союзы
Массовая культура

Спорт: Борьба
Массовая культура

Массовая культура: Юмор и сатира

Дом и сад: Дизайн интерьера

Транспорт
Спорт: Автогонки
События и достопримечательности: Спортивные события

Спорт: Рыбалка

Массовая культура: Юмор и сатира

Массовая культура: Юмор и сатира
Семья и отношения

Личные финансы
Карьера

Дом и сад: Дизайн интерьера

Стиль и красота: Женская мода

Массовая культура: Юмор и сатира
Транспорт

Массовая культура: Юмор и сатира

Массовая культура: Юмор и сатира

Спорт
Массовая культура: Юмор и сатира

Массовая культура: Отношения знаменитостей

Массовая культура: Юмор и сатира

Массовая культура: Юмор и сатира

Спорт: Рыбалка

Массовая культура: Юмор и сатира
Семья и отношения

Массовая культура
Спорт

Музыка и аудио: Комедия и стендап (Музыка и аудио)
События и достопримечательности: Концерты и музыкальные мероприятия

Массовая культура: Скандалы знаменитостей

Массовая культура: Юмор и сатира

Религия и духовность: